## MakeMyTrip

#### Importing all requried libraries

In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.feature_selection import mutual_info_classif

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV




import warnings
warnings.filterwarnings('ignore')

#### Data Collection

Scrapping from makeMyTrip

In [2]:
chrome_options = webdriver.ChromeOptions()
agents = ["Firefox/66.0.3","Chrome/95.0.4638.54","Edge/16.16299"]
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

today = datetime.datetime.strftime(datetime.datetime.today() , '%d/%m/%Y')
_90daysfromnow=[]

for i in range(90):
    oneDayLater = datetime.datetime.today() + datetime.timedelta(days = i)
    _90daysfromnow.append(datetime.datetime.strftime(oneDayLater , '%d/%m/%Y'))

# list of Indian city airport code 
city = ['DEL','BLR','BOM','PNQ','HYD','AMD','BUP','BHO','BBI','GAU','IXC','DED','DBD','GAY','GWL','JLR','KNU','KTU','MZU','MYQ','CCU','IXZ','IXG','NAG','S','VNS','BHJ','ATQ']
src_dst=[]

for rnd in range(50):
    _source=random.choice(city)
    _destination=random.choice(city)
    if _source!=_destination:
        srcDst=_source+'-'+_destination
        src_dst.append(srcDst)
uniquesrcndst=set(src_dst) # for unique source and destination code
traveldate=[]
airlines=[]
source=[]
sourcendestination=[]
departureAndarival=[]
durationAndRoute=[]
prices=[]
count=0
for date in _90daysfromnow:
    for sd in uniquesrcndst:
        searchitineraray=sd+'-'+date
        
        url='https://www.makemytrip.com/flight/search?itinerary='+searchitineraray+'&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng'
        driver.get(url)
        time.sleep(3)
        
        try:
            
            airline=driver.find_elements_by_xpath('//*[@class="boldFont blackText airlineName"]')
            for al in airline:
                airlines.append(al.text)
                
            for travelDate in range(len(airline)):
                traveldate.append(date)
            
            
            srcndst=driver.find_elements_by_xpath('//*[@class="darkText"]')
            for sd in srcndst:
                sourcendestination.append(sd.text)
            
            
            # departure and arival time
            departureandarival=driver.find_elements_by_xpath('//*[@class="blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"]')
            for da in departureandarival:
                departureAndarival.append(da.text)
                
            
            # travel durations and route
            durationAndroute=driver.find_elements_by_xpath('//*[@class="stop-info flexOne"]')
            for dnr in durationAndroute:
                durationAndRoute.append(dnr.text.split('\n'))
            
            
            
            #fare
            price=driver.find_elements_by_xpath('//*[@class="blackText fontSize18 blackFont white-space-no-wrap"]')
            for pr in price:
                prices.append(pr.text)
      
        
        except NoSuchElementException:
            pass
    count+=1
    if count==10001:
        break

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=95.0.4638.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00813AB3+2505395]
	Ordinal0 [0x007AAE41+2076225]
	Ordinal0 [0x006B2498+1057944]
	Ordinal0 [0x006B4BC4+1067972]
	Ordinal0 [0x006B4A8E+1067662]
	Ordinal0 [0x006B4CF0+1068272]
	Ordinal0 [0x006D8600+1213952]
	Ordinal0 [0x006F59F3+1333747]
	Ordinal0 [0x006D3676+1193590]
	Ordinal0 [0x006F5ADA+1333978]
	Ordinal0 [0x00705168+1397096]
	Ordinal0 [0x006F58BB+1333435]
	Ordinal0 [0x006D23E4+1188836]
	Ordinal0 [0x006D323F+1192511]
	GetHandleVerifier [0x0099CB36+1554566]
	GetHandleVerifier [0x00A44A0C+2242396]
	GetHandleVerifier [0x008A0E0B+523099]
	GetHandleVerifier [0x0089FEB0+519168]
	Ordinal0 [0x007B02FD+2097917]
	Ordinal0 [0x007B4388+2114440]
	Ordinal0 [0x007B44C2+2114754]
	Ordinal0 [0x007BE041+2154561]
	BaseThreadInitThunk [0x77166739+25]
	RtlGetFullPathName_UEx [0x77D18AFF+1215]
	RtlGetFullPathName_UEx [0x77D18A

In [3]:
route=[]
for dr in durationAndRoute:
    for r in dr:
        route.append(r)

In [4]:
routevia=route[1::2]
duration=route[::2] 
source=sourcendestination[::2]
destination=sourcendestination[1::2]
departure=departureAndarival[::2]
arival=departureAndarival[1::2]
print(len(airlines))
print(len(source))
print(len(destination))
print(len(arival))
print(len(departure))
print(len(routevia))
print(len(prices))
print(len(duration))

4419
4419
4419
4404
4405
4226
4398
4226


In [5]:
df=pd.DataFrame()
df['Airline']=airlines[:4200]
df['Source']=source[:4200]
df['Duration']=duration[:4200]
df['Departure']=departure[:4200]
df['Arival']=arival[:4200]
df['Destination']=destination[:4200]
df['Travel Date']=traveldate[:4200]
df['Route']=routevia[:4200]
df['Fare']=prices[:4200]

In [6]:
df.to_csv('mmt.csv')

In [7]:
df.head(200)

Airline  Source   Duration Departure Arival Destination  \
0             Go First    Pune  07 h 25 m     15:30  22:55    Varanasi   
1     IndiGo, Go First    Pune  06 h 40 m     16:15  22:55    Varanasi   
2               IndiGo    Pune  05 h 55 m     14:00  19:55    Varanasi   
3    Vistara, Go First    Pune       06 h     16:55  22:55    Varanasi   
4    Vistara, Go First    Pune  09 h 45 m     13:10  22:55    Varanasi   
..                 ...     ...        ...       ...    ...         ...   
195             IndiGo  Kanpur  01 h 05 m     13:45  01:00   Hyderabad   
196   Spicejet, IndiGo    Pune  01 h 05 m     08:25  14:20      Bhopal   
197   Go First, IndiGo    Pune       01 h     11:15  19:20      Bhopal   
198             IndiGo    Pune  01 h 05 m     13:45  19:20      Bhopal   
199   Spicejet, IndiGo    Pune       01 h     08:25  19:20      Bhopal   

    Travel Date                 Route      Fare  
0    01/11/2021  1 stop via New Delhi  ₹ 12,021  
1    01/11/2021  1 stop via New Delhi  ₹ 13,172  
2    01/11/2021  1 stop via Bengaluru  ₹ 13,345  
3    01/11/2021  1 stop via New Delhi  ₹ 13,488  
4    01/11/2021  1 stop via New Delhi  ₹ 13,488  
..          ...                   ...       ...  
195  02/11/2021              Non stop   ₹ 4,499  
196  02/11/2021              Non stop  ₹ 12,332  
197  02/11/2021              Non stop  ₹ 14,118  
198  02/11/2021              Non stop  ₹ 14,306  
199  02/11/2021              Non stop  ₹ 14,432  

[200 rows x 9 columns]